In [1]:
import re
import pickle
import time

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import tensorflow_hub as hub

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [3]:
# Загрузим датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df.shape

(497819, 4)

In [6]:
# Загрузим предобученный universal-sentence-encoder
use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [7]:
# Сделаем списки названий компаний.
df_brand_names = pd.DataFrame(pd.concat([df.name_1, df.name_2], axis=0).drop_duplicates(), columns=['name'])
brand_names = df_brand_names.name.tolist()

In [8]:
# Получим эмбеддинги из universal-sentence-encoder.
brand_embeddings = use(brand_names)
top_k = 10

In [9]:
# Функция для нахождения топ 10 похожих на исходное названий.
def find_similar_name(sentence, embeddings, top_k, is_main):
    sentence_embedding = use([sentence])
    similarity = cosine_similarity(sentence_embedding, embeddings)
    top = sorted(enumerate(*similarity), key=lambda x: x[1], reverse=True)
    similar_names = [brand_names[x[0]] for x in top[:top_k]]
    
    if is_main == 0:
        for name in similar_names:
            print(name)
    elif is_main == 1:
        return similar_names

In [10]:
# Время начала обработки поиска. 
start_time = time.time()

# Найдём топ 10 похожих названий для компании Mediterranean Textile Co, которая была в датасете.
is_main = 0
print(brand_names[80])
print()
find_similar_name(brand_names[80], brand_embeddings, top_k, is_main)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Mediterranean Textile Co.

Mediterranean Textile Co.
Zamoung Textile Co.
Torreh Textile Co.
Kavir Semnan Textile Co.
Ardakan Textile Co.
Ferdows Textile Co.
Hanam Textile Co.
Mediterranean Textile
Razi Abhar Textile Co.
Alroubaia Textile Co Fourtex

Время обработки поиска =  0.621


In [11]:
# Вдими саму компанию и названия с такими же словами.

In [12]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь получим эмбеддинг и найдём похожие названия для строки, которой не было в датасете.
my_text_test_1 = 'Darth Vader is a great Sith Lord'
print(my_text_test_1)
print()

find_similar_name(my_text_test_1, brand_embeddings, top_k, is_main)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Darth Vader is a great Sith Lord

Hussein Hussein Hezam Mohmmed
Sonic Fluid Power Systems (Pvt.)
Atat Tasimacilik Cozumleri A.S.
Abdullah Hussain Ali
The Cato Corporation
Ashoka Inc.
Sound Merch Bmg (Neutral)
Hyundai Power Transformers Usa
Sika S.A. Chile
Kia Chile S.A.

Время обработки поиска =  0.108


In [13]:
# Интересно, что к Дарту Вейдеру модель вывела названия, где фигурировало слово Power. Аж два названия.
# "Atat" возможно из-за похожести на AT-AT (Имперский шагоход). 
# И "Ashoka Inc." из-за похожести  "Ashoka" на "Ahsoka". 
# Потому что Ahsoka Tano это ученица Вейдера, времён, когда его еще Энакином звали.

In [14]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь получим эмбеддинг и найдём похожие названия для строки, похожей, на то, что было в датасете.
# В датасете было название "Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V."
# Посмотрим, что на него выдаст модель.

my_text_test_2 = 'Well Aero System Lab Industrial Craft'
print(my_text_test_2)
print()

find_similar_name(my_text_test_2, brand_embeddings, top_k, is_main)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Well Aero System Lab Industrial Craft

Golden Wheel Industrial Mfg. Corp
Titan Rubber Industrial Mfg. Corp
Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Mova Industrial S.A.C.
Newpro Industrial Mfg. Corp
Aero Club
Erbus Industrial Eireli
Aero Traders
Distribuidora Industrial Avanzada Sa De Cv
Pegsa Industrial Sac

Время обработки поиска =  0.106


In [15]:
# Видим, что нашлись названия компаний с похожими словами:  "Industrial", "Aero", "Aerospace", "Laboratory".
# Время обработки поиска 0.1-0.3 секунды.

In [18]:
# Функция позволит вводить строку для поиска с клавиатуры.
def main():
    while True:
        comp_name = input("Введите название компании:\n\n").rstrip()
        if comp_name == "exit":
            break
        k = 10
        is_main = 1
        start_time = time.time()
        
        try:
            top_comp = find_similar_name(comp_name, brand_embeddings, k, is_main) 
        except ValueError:
            print ("Похожих компаний нет в списке или введены кракозябры\n\n")
            continue
        print (f"Топ {k} похожих компаний:\n\n")
        for i, comp in enumerate(top_comp):
            print (f"{i + 1}: {comp}")
        print("\n")
        
        end_time = time.time()

        print("Время обработки поиска = ", round(end_time-start_time, 3))
        print()

In [19]:
main()

Введите название компании:

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Топ 10 похожих компаний:


1: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
2: Honeywell Aerospace De Mexico Sa De Cv
3: Boss Laboratories S De Rl De Cv
4: Honeywell Productos Automotrices Sa De Cv
5: Psc Maquiladora S De Rl De Cv
6: Tpi Composites S. De R.L. De C.V.
7: Tubac Mfg. S De Rl De Cv
8: Encore Aerospace Mexico S. De R.L. De C.V.
9: Southwest Fabricators S De R.L. De C.V.
10: Kauffman Mfg., S. De R.L. De C.V.


Время обработки поиска =  0.105

Введите название компании:

Goldet Steel Industrie
Топ 10 похожих компаний:


1: Emirates Steel Industries
2: Lodhia Steel Industries Ltd.
3: Hulas Steel Industries Ltd.
4: Sujan Industries
5: Jayam Industries
6: Chemlink Industries
7: Swastic Auto Industries
8: Gehr Industries
9: Jatin Industries
10: Bindoo Industries


Время обработки поиска =  0.098

Введите название компании:

ghh hjghk ghkghk4565675 878878787
Топ 10 похожих к

In [20]:
is_main = 1

# Возьмём из датасета те названия, что имеют схожие.
df_duplicate = df[df['is_duplicate'] == 1]

name_1_duplicate = list(df_duplicate['name_1'])
name_2_duplicate = list(df_duplicate['name_2'])

# Создадим словарь, чтобы можно было найти дубль для каждого названия из списка дубликатов.
duplicate_dictionary = dict(zip(name_1_duplicate+name_2_duplicate, name_2_duplicate+ name_1_duplicate))


# Функция подсчёта метрики ранжирования.
def search_acc_metric(names):
    d_s = []
    for name in names:
        try:
            duplicate_dictionary[name]
            d_s.append(1)
        except KeyError:
            d_s.append(0)
    
    k = len(names)
    if 1 in d_s:
        w_i = [max(-0.2*i+1,0) for i in range(k)]
        result = min(sum([x*y for x, y in zip(w_i, d_s)]), 1)
    else:
        w_i = [0.2*i for i in range(1, k+1)]
        result = max(1 - sum(w_i), 0)

    return result


In [21]:
df_duplicate.head(10)

,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
1786,1787,Pt Bridgestone Tire Indonesia,Bridgestone Do Brasil Industria E Comenrcio Ltda,1
1798,1799,Brenntag Peru S.A.C.,Brenntag Group,1
1962,1963,"Carlisle Coatings & Waterproofing, Inc.",Carlisle Coatings & Wtrprfng,1
2054,2055,Pt Bridgestone Tire Indonesia,"Thai Bridgestone Co., Ltd.",1
2772,2773,Onduline Polska Sp. z o.o.,Onduline S.A.,1


In [22]:
# Возьмём название у которого есть похожее и посмотрим значение метрики.
companies_names = find_similar_name("Carlisle Coatings & Waterproofing, Inc.", brand_embeddings, top_k, is_main)

for companies in companies_names:
    print(companies)

Carlisle Coatings & Waterproofing, Inc.
CARLISLE COATINGS
Carlisle Coatings & Wtrprfng
Devan Sealants Inc.
Arkema Inc. Coatings & Resins 39
Carlisle Construction Materials
Dsm Coating Resins Inc.
Ticona Polymers Inc.
Ol Roofing Supplies Inc.
Monico Alloys Inc.


In [23]:
# Теперь посчитаем метрику   
print(search_acc_metric(companies_names))

1


In [24]:
# Возьмём название у которого нет похожего и посмотрим значение метрики.
companies_names = find_similar_name("Powermax Rubber Factory", brand_embeddings, top_k, is_main)

for companies in companies_names:
    print(companies)

Powermax Rubber Factory
Basant Rubber Factory
Pioneer Rubber Factory
Rachana Rubber
Le Caoutchouc Mfg.
Mountville Mills Rubber
Sumitomo Rubber Inds
Neelkanth Rubber Mills
Triveni Rubber
Kayel Rubber Products


In [25]:
# Теперь посчитаем метрику   
print(search_acc_metric(companies_names))

0.0
